# Dropar colunas que nao vou usar

In [8]:
import pandas as pd

In [9]:
try:
    df = pd.read_csv(
        '../data/tasks.csv',
        encoding='utf-8',
        sep=",",
        quotechar='"',
        on_bad_lines='skip',      # ignora linhas corrompidas
        dtype=str,               # carrega tudo como string, depois ajusta
        engine='c'          # usa engine python que é mais tolerante
    )
    print(f"CSV carregado com engine C! {len(df)} linhas encontradas.")

except Exception as e:
    print(f"Erro ao ler CSV: {e}")

CSV carregado com engine C! 7232 linhas encontradas.


In [10]:
# Análise básica de valores nulos por coluna
null_counts = df.isnull().sum()
print("Valores nulos por coluna:")
print(null_counts)

Valores nulos por coluna:
SUMMARY                             0
DTSTART                             0
DTEND                               2
DUE                              7232
NOTES                            6657
ATTENDEE                         7193
LOCATION                         5457
PRIORITY                         7232
URL                              7232
CALENDAR                            0
UID                              7232
ORGANIZER                        7204
CATEGORIES                       7232
DURATION                            2
REPLACES RECURRENT EVENT FROM    7145
CANCELLED                        7047
CREATED                             0
dtype: int64


In [11]:
cols_to_drop = [
    'DUE', 'NOTES', 'ATTENDEE', 'LOCATION', 'PRIORITY', 'URL', 'UID',
    'ORGANIZER', 'CATEGORIES', 'REPLACES RECURRENT EVENT FROM',
    'CANCELLED', 'CREATED'
]
df = df.drop(columns=cols_to_drop)
df.head()

,SUMMARY,DTSTART,DTEND,CALENDAR,DURATION
0,🍳,07/01/2025 08:00,07/01/2025 08:20,Alimentação,0.33333333333333
1,🧋,07/01/2025 10:00,07/01/2025 10:05,Alimentação,0.083333333333333
2,🍽️,07/01/2025 12:30,07/01/2025 13:00,Alimentação,0.5
3,🥪,07/01/2025 16:00,07/01/2025 16:15,Alimentação,0.25
4,🍌,07/01/2025 18:00,07/01/2025 18:05,Alimentação,0.083333333333333


In [12]:
# Converter as colunas DTSTART e DTEND para datetime
df['DTSTART'] = pd.to_datetime(df['DTSTART'], format='%d/%m/%Y %H:%M', errors='coerce')
df['DTEND'] = pd.to_datetime(df['DTEND'], format='%d/%m/%Y %H:%M', errors='coerce')

# Definir o limite máximo como o último dia de julho de 2025
limite = pd.Timestamp('2025-07-18 23:59:59')

# Mostrar todos os SUMMARY que estão passando do limite antes de filtrar
summary_passando_limite = df[(pd.to_datetime(df['DTSTART'], errors='coerce') > limite) | (pd.to_datetime(df['DTEND'], errors='coerce') > limite)]
print("SUMMARY que estão passando do limite de data:")
print(summary_passando_limite['SUMMARY'].unique())

print(f"Número de linhas antes o filtro de datas: {len(df)}")

# Filtrar linhas onde ambas as datas são menores ou iguais ao limite
df = df[(df['DTSTART'] <= limite) & (df['DTEND'] <= limite)]

print(f"Número de linhas após o filtro de datas: {len(df)}")

SUMMARY que estão passando do limite de data:
['🍳' '🧋' '🍽️' '🥪' '🍌' 'INF1410 - Gerenc Proj Inf'
 'INF1608 - Análise Numérica' 'ADM1019 - Intr. Finanças'
 'INF1629 - P. Eng. de Software' 'Parabéns!' 'Psicólogo' 'Dr Bruno'
 'Academia' 'Morgan Jay' 'Aniversário da minha irmã'
 'MAT4162 - Cálculo II' 'INF1314 - Startup IA' 'Formatura Lu' 'M3']
Número de linhas antes o filtro de datas: 7232
Número de linhas após o filtro de datas: 4464


In [13]:
df.to_csv('../data/new_tasks.csv', index=False, encoding='utf-8')